In [1]:
import pandas as pd
import geopandas as gpd
import os

from tqdm.notebook import tqdm


import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import datetime
from datetime import date, timedelta
e_date = datetime.datetime.strptime('2020-06-20', '%Y-%m-%d')
i_date = datetime.datetime.strptime('2020-04-01', '%Y-%m-%d')

data_path = os.path.join('OD')

In [2]:
delta = e_date - i_date

target_days = []
for i in range(delta.days + 1):
    day = i_date + timedelta(days=i)
    target_days.append(day)

def read_total_trips_fn(source_type):
    total_trips = None
    for date in tqdm(target_days):
        od_date= pd.read_csv(os.path.join(data_path, 'date_{}_OD_{}.csv'.format(source_type,date.strftime('%Y-%m-%d'))), 
                            index_col=0)
        if total_trips is not None:
            total_trips += od_date['total']
        else:
            total_trips= od_date['total']
    return total_trips

In [3]:
ine_total = read_total_trips_fn('INE')

  0%|          | 0/81 [00:00<?, ?it/s]

In [4]:
twt_totals= read_total_trips_fn('TWT')

  0%|          | 0/81 [00:00<?, ?it/s]

In [24]:
ine_total_df= ine_total.to_frame()
ine_total_df.head()

,total
001A,53132
001B,95526
001M,0
001S,765
001V,49980


In [25]:
twt_total_df= twt_totals.to_frame()
twt_total_df.head()

,total
001A,18
001B,53
001M,9
001S,30
001V,16


In [20]:
areas = pd.read_csv(os.path.join('OD', 'areas_pop.csv'),index_col='ID_GRUPO')
areas.head()

,OBJECTID,SUM_POB_AS,Shape_Leng,MOMENTO_TE,MOMENTO__1,MOMENTO__2,MOMENTO__3,MOMENTO__4,MOMENTO__5,MOMENTO__6,...,MOMENTO_16,MOMENTO_17,MOMENTO_18,MOMENTO_19,MOMENTO_20,MOMENTO_21,MOMENTO_22,MOMENTO_23,Shape_Le_1,Shape_Area
ID_GRUPO,,,,,,,,,,,,,,,,,,,,,
001A,1,7033.0,35771.338209,92.404465,79.893962,75.922711,77.236382,106.715099,181.03,113.084606,...,115.57,137.40,137.59,138.68,130.81,130.73,100.0,81.00,45468.799036,8.237841e+07
001B,2,12588.0,63295.634626,95.902610,81.079163,74.227978,76.986806,95.866571,109.19,94.763138,...,79.17,83.12,85.11,93.65,93.76,99.88,100.0,89.47,84763.998793,1.452931e+08
001M,3,4550.0,120439.774794,93.464337,80.622183,77.888138,81.000670,94.148174,139.27,102.273435,...,82.36,101.69,105.75,108.32,107.60,105.01,100.0,85.95,157744.983867,4.756526e+08
001S,4,6035.0,262576.084345,97.639286,89.265089,84.974096,86.644783,97.604245,120.61,102.609369,...,85.47,102.54,105.47,104.11,102.95,101.68,100.0,91.46,332209.730375,1.436231e+09
001V,5,5495.0,18204.294363,93.129927,80.389465,76.114306,78.586172,94.987062,129.98,101.709716,...,85.70,96.59,101.75,106.48,104.26,106.06,100.0,79.96,23413.453710,1.126422e+07


In [30]:
areas_pop = areas['SUM_POB_AS'].to_frame()
areas_pop.head()

,SUM_POB_AS
ID_GRUPO,
001A,7033.0
001B,12588.0
001M,4550.0
001S,6035.0
001V,5495.0


In [31]:
areas_pop['twt_total']= twt_total_df['total']
areas_pop['ine_total']= ine_total_df['total']

In [50]:
areas_pop[(areas_pop['ine_total']>0) & (areas_pop['twt_total']> 0)].corr()

,SUM_POB_AS,twt_total,ine_total
SUM_POB_AS,1.000000,0.177296,0.835887
twt_total,0.177296,1.000000,0.137026
ine_total,0.835887,0.137026,1.000000


In [91]:
lite_df = areas_pop[(areas_pop['ine_total']>0) & (areas_pop['twt_total']> 1)& (areas_pop['twt_total']< 2000)]

In [92]:
lite_df.corr()

,SUM_POB_AS,twt_total,ine_total
SUM_POB_AS,1.000000,0.350524,0.836121
twt_total,0.350524,1.000000,0.267969
ine_total,0.836121,0.267969,1.000000


In [56]:
areas_pop[(areas_pop['SUM_POB_AS']>2000)].corr()

,SUM_POB_AS,twt_total,ine_total
SUM_POB_AS,1.000000,0.164860,0.841462
twt_total,0.164860,1.000000,0.116273
ine_total,0.841462,0.116273,1.000000


In [51]:
areas_pop[(areas_pop['twt_total']> 0)].corr()

,SUM_POB_AS,twt_total,ine_total
SUM_POB_AS,1.000000,0.178487,0.837389
twt_total,0.178487,1.000000,0.138461
ine_total,0.837389,0.138461,1.000000


In [33]:
areas_pop.corr()

,SUM_POB_AS,twt_total,ine_total
SUM_POB_AS,1.000000,0.164923,0.841550
twt_total,0.164923,1.000000,0.116347
ine_total,0.841550,0.116347,1.000000


In [36]:
areas_pop.loc[['58MA','156B','34CZ','165V','13VI']].corr()

,SUM_POB_AS,twt_total,ine_total
SUM_POB_AS,1.000000,0.107268,-0.914201
twt_total,0.107268,1.000000,-0.243229
ine_total,-0.914201,-0.243229,1.000000
